In [ ]:
import re
import logging
import hashlib
from utils import get_context, get_response
from inference import fetch as fetch_inflection

# **Automated Specification Validation using Inflection AI**

This notebook demonstrates how to **validate specifications** using AI-driven approaches. The goal is to ensure **compliance and correctness** in structured documents by leveraging **automated validation** techniques.

## **Overview**

- Implements **AI-based validation** for structured specifications.
- Utilizes a **prompting-based model** to analyze document content.
- The model performs the following tasks:
  - Extracts **key compliance criteria** from specification documents.
  - Identifies **potential inconsistencies and errors**.
  - Outputs structured validation reports for **further review**.
- Uses a **state-of-the-art Inflection AI LLM** optimized for analysis and validation.

In [ ]:
legacy_api = False # True if using the old API, False if using the new OpenAI Compatible API

In [ ]:
system_instruction_prompt_example_1 = """
You are an expert Python test writer with a deep understanding of software testing principles, including unit testing, edge case analysis, and test-driven development (TDD). Your goal is to write high-quality, robust unit tests for the provided Python code, ensuring correctness, reliability, and full specification coverage.

# Your Responsibilities
1. You will receive a Python code implementation from the user.
2. You will also receive a  set of specifications that describe the expected behavior of the code.
3. Based on these inputs, you must generate comprehensive unit tests to validate the implementation.

# Test Writing Guidelines:
- Coverage: Ensure that all key functionalities described in the specifications are tested.
- Edge Cases: Identify and include edge cases, boundary values, and potential failure scenarios.
- Assertions: Use clear and meaningful assertions to verify expected behavior.
- Mocking & Fixtures: If applicable, use mocking (e.g., `unittest.mock`) and test fixtures to isolate test dependencies.
- Best Practices: Follow Python testing best practices using frameworks like `unittest` or `pytest`.
- Readability & Maintainability: Write clean, well-structured tests with meaningful test names and comments.

# Output Format
- Provide well-documented Python test code.
- Use descriptive test function names that reflect the behavior being tested.
- Follow a structured format for readability and maintainability.

Your goal is to ensure that the provided code is thoroughly tested, meets all requirements, and is resilient to edge cases.
"""


system_instruction_prompt_example_2 = """
You are a compliance expert with a deep understanding of regulatory requirements and best practices in the field. Your goal is to analyze the provided code and provided specification to ensure that they meet all necessary compliance standards.

# Your Responsibilities
1. You will receive a Python code implementation from the user.
2. You will also receive an industry compliance specification that describes the expected behavior of the code.
3. Based on these inputs, you must analyze the code and specification to identify any compliance issues.

# Compliance Analysis Guidelines:
- Identify if the code meets the specifications: yes or no
- Provide an explanation of the compliance status: explain why the code is compliant or not
- Provide recommendations for improvement: suggest changes to ensure compliance

# Output Format - XML
You will respond using XML tags. You don't need to provide explanation or any other information, just return the extracted parts within the appropriate XML tags.

# Format of the Output:
<parts>
    <compliant>yes or no</compliant>
    <explanation>explanation of the compliance status</explanation>
    <recommendations>recommendations for improvement</recommendations>
</parts>
"""


## Test Scenario: Test generation based on specifications

In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
async def test_specs_validation():
    print("Starting test: test_specs_validation")
    print("+*"*20)
    user_function = """
    def divide_numbers(numerator, denominator):
        try:
            result = numerator / denominator
            return result
        except ZeroDivisionError:
            return 'Error: Division by zero is undefined.'
        except OverflowError:
            return 'Error: Number too large to compute.'
    """

    user_specifications = """
    - Write unit tests using pytest.
    - Parameterize test cases for efficiency.
    - Ensure all edge cases are tested.
    - Validate exception handling for invalid inputs.
    """

    query = f"User function: {user_function}\nUser specifications: {user_specifications}"
    context = get_context(system_instruction_prompt_example_1, query, legacy_api=legacy_api)
    response = await fetch_inflection(context, legacy_api=legacy_api)

    print(f"{color.BOLD} Response: {color.END} {response}")
    print("+*"*20)

    print("Test completed successfully! 🙌")

# Run the test
await test_specs_validation()

## Test Scenario: Software compliance check of the test

In [ ]:
def extract_software_compliance_rules(file_path):
    """Extracts Software Compliance Rules from a given compliance document file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Regular expression to match the rules
    pattern = re.compile(r'\d+\.\s([A-Za-z\s\-]+):\s(.*?)$', re.MULTILINE)
    
    # Extract matches
    rules = pattern.findall(text)
    
    # Format the results into a list of strings
    formatted_rules = [f"{num}. {title}: {desc}" for num, (title, desc) in enumerate(rules, start=1)]
    
    return formatted_rules

# Example usage:
file_path = "compliance_doc.txt"
rules = extract_software_compliance_rules(file_path)
for rule in rules:
    print(rule)

In [ ]:
def secure_transaction_log(transaction_details):
    try:
        # Encrypt transaction details using SHA-256
        transaction_hash = hashlib.sha256(transaction_details.encode()).hexdigest()
        
        # Store securely in log
        log_path = "secure_transaction.log"
        with open(log_path, "a") as log_file:
            log_file.write(f"{transaction_hash}\n")
        
        return "Transaction logged securely."
    except Exception as e:
        logging.error("Error in transaction logging: %s", str(e))
        return "Transaction logging failed."

# Example usage
print(secure_transaction_log("User A transferred $500 to User B"))

In [ ]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
for rule in rules:
    query = f"User function: {str(secure_transaction_log)}\nIndustry compliance specification: {rule}"
    context = get_context(system_instruction_prompt_example_2, query, legacy_api=legacy_api)
    response = await get_response(context, ["compliant", "explanation", "recommendations"], legacy_api=legacy_api)
    
    print(f"{color.BOLD} Compliance Rule: {color.END} {rule} ")
    print(f"{color.BOLD} Is this code compliant?: {color.END} {response["compliant"]}")
    print(f"{color.BOLD} Explanation: {color.END} {response["explanation"]}")
    print(f"{color.BOLD} Recommendations: {color.END} {response["recommendations"]}")
    print("+*"*20)